## Importing the LonelyBoy Library (github.com/insert-generic-name-here/lonelyboy)

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser('~'), 'Documents/Coding/Python'))
# sys.path

from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp
from lonelyboy.geospatial import group_patterns_v2 as gsgp## Importing the LonelyBoy Library (github.com/insert-generic-name-here/lonelyboy)

## Importing all other Essential Libraries
#### (DO NOT FORGET TO EXECUTE THE FUNCTIONS IN THE BOTTOM CELLS)

In [3]:
import psycopg2
import numpy as np
import configparser
import pandas as pd
import geopandas as gpd
import contextily as ctx
from random import choice
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.base import clone
from sklearn.cluster import DBSCAN, KMeans, MeanShift
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from shapely.geometry import Point, LineString, shape
from haversine import haversine
from datetime import datetime, timedelta

In [4]:
from multiprocessing import cpu_count, Pool
from functools import partial
import datetime

## Import Libraries for Visualizations

In [5]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import PyQt5
import matplotlib.pyplot as plt
from matplotlib import style;  style.use('ggplot')
get_ipython().magic('matplotlib qt')
# get_ipython().magic('matplotlib inline')

## Get & Plot a Random Vessel's Trajectory

In [5]:
%%time
import random

mmsis = pd.read_pickle('data/pkl/mmsi_list.pckl')
mmsi_selected = random.choice(random.choice(mmsis))

csv_iter = gspp.read_csv_generator('data/csv/nari_dynamic.csv')
chunk = pd.concat((chunk[chunk['mmsi'] == mmsi_selected] for chunk in csv_iter), ignore_index=True)
chunk = gspp.gdf_from_df(chunk, crs={'init':'epsg:4326'})

print (mmsi_selected)
chunk.head()

538005408
CPU times: user 10.7 s, sys: 1.25 s, total: 11.9 s
Wall time: 12 s


In [73]:
chunk.sort_values(['ts'], inplace=True)

In [6]:
gsplt.map_plot(chunk, column='ts', figsize=(10,10), color=[None], legend=True)

## Test Plots

In [9]:
csv_iter = gspp.read_csv_generator('data/csv/mmsi/205067000.csv')
chunk = pd.concat((chunk[chunk['mmsi'] == 205067000] for chunk in csv_iter), ignore_index=True)
chunk = gspp.gdf_from_df(chunk, crs={'init':'epsg:4326'})

ports = pd.read_pickle('data/pkl/ports_raw.pkl')
ports = gspp.create_port_bounds(ports, port_radius=2000)

In [ ]:
gsplt.map_plot(chunk, ports, column='ts', color=[None, 'r'], figsize=(10,10), legend=True)

In [28]:
tmp = chunk.ts.diff()/(60*60*24)
plt.title('Trajectory Time Delta Plot')
plt.xlabel('Record Index')
plt.ylabel('Time Delta (in Days)')
tmp.plot()

## Test Code

  * ## Calculating Velocity

In [7]:
chunk_vel = gspp.calculate_velocity(chunk, smoothing=False).sort_values(['ts'])

/home/andretri7/anaconda3/envs/gis/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/andretri7/anaconda3/envs/gis/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/andretri7/anaconda3/envs/gis/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docum

In [8]:
plt.figure(1)
plt.title('velocity')
plt.plot(chunk_vel.velocity.values)

  * ## Calculating Acceleration

In [10]:
# window = 3
# center=False

chunk_vel['acceleration'] = chunk_vel.velocity.diff().divide(chunk_vel.ts.diff())
# chunk_vel['acceleration'] = chunk_vel.velocity.diff(-1)
# chunk_vel['acceleration'] = chunk_vel['acceleration'].rolling(window, center=center).mean().bfill().ffill()
chunk_vel.sort_values(['ts'], inplace=True)

chunk_vel = chunk_vel.loc[chunk_vel['mmsi'] != 0]
chunk_vel.dropna(subset=['mmsi', 'geom'], inplace=True)
chunk_vel.fillna(0, inplace=True)

plt.figure(2)
plt.title('acceleration')
plt.plot(chunk_vel.acceleration.values)

  * ## Calculating Course

In [106]:
def calculate_course(point1, point2):
    '''
        Calculating initial bearing between two points
        (see http://www.movable-type.co.uk/scripts/latlong.html)
    '''
    lon1, lat1 = point1[0], point1[1]
    lon2, lat2 = point2[0], point2[1]

    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)
    numerator = np.sin(dlon) * np.cos(lat2)
    denominator = (
        np.cos(lat1) * np.sin(lat2) -
        (np.sin(lat1) * np.cos(lat2) * np.cos(dlon))
    )

    theta = np.arctan2(numerator, denominator)
    theta_deg = (np.degrees(theta) + 360) % 360
    return theta_deg

In [112]:
chunk_vel2 = chunk_vel.copy()
chunk_vel2['bearing'] = np.nan
chunk_vel2['current_loc'] = chunk_vel2.geom.apply(lambda x: (x.x,x.y))
chunk_vel2['next_loc'] = chunk_vel2.geom.shift(-1)
chunk_vel2 = chunk_vel2[:-1]
chunk_vel2.loc[:,'next_loc'] = chunk_vel2.next_loc.apply(lambda x : (x.x,x.y))

chunk_vel['bearing'] = chunk_vel2[['current_loc', 'next_loc']].apply(lambda x: calculate_course(x[0], x[1]), axis=1)

In [114]:
chunk_vel = chunk_vel.bfill().ffill()
chunk_vel

,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom,velocity,acceleration,bearing
0,538002700,0.0,-4.0,13.9,27.0,23,-6.285498,47.935833,1458794899,POINT (-6.285498 47.935833),15.533785,0.000508,332.013944
1,538002700,0.0,18.0,13.8,27.0,22,-6.272498,47.952330,1458795192,POINT (-6.272498000000001 47.95233),15.533785,0.000508,333.266589
2,538002700,0.0,-12.0,13.9,28.0,23,-6.271998,47.953000,1458795204,POINT (-6.2719984 47.953),15.533785,0.000508,335.796200
3,538002700,0.0,20.0,13.8,28.0,23,-6.270999,47.954500,1458795228,POINT (-6.2709985 47.9545),15.570336,0.000508,332.460378
4,538002700,0.0,11.0,13.8,27.0,23,-6.257165,47.972000,1458795537,POINT (-6.257165000000001 47.972),15.426698,0.000353,352.448189
5,538002700,0.0,9.0,12.2,27.0,23,-5.817165,48.526333,1458806004,POINT (-5.817165 48.526333),14.558153,0.000325,352.435184
6,538002700,0.0,16.0,12.2,26.0,24,-5.816498,48.527170,1458806021,POINT (-5.8164983 48.52717),13.246509,-0.025901,0.000000
7,538002700,0.0,22.0,12.2,26.0,25,-5.816498,48.527332,1458806024,POINT (-5.8164983 48.527332),12.902234,-0.063999,6.435245
8,538002700,0.0,18.0,11.7,61.0,57,-5.540665,48.807835,1458812292,POINT (-5.540665 48.807835),14.701001,-0.063876,18.915057
9,538002700,0.0,-12.0,11.7,60.0,57,-5.540165,48.808000,1458812298,POINT (-5.540165 48.808),16.303811,0.050888,17.612297


In [119]:
# chunk_vel.bearing.plot.hist()
# chunk_vel.course.plot.hist()

## Old (and maybe useless) Code

In [58]:
import math
#                   sin(lat2) - sin(lat1)*cos(d)
#    Bearing = acos ----------------------------
#                      sin(d) * cos(lat1)

chunk_vel2 = chunk_vel.copy()

chunk_vel2['bearing'] = np.nan

chunk_vel2['current_loc'] = chunk_vel2.geom.apply(lambda x: (x.x,x.y))
chunk_vel2['next_loc'] = chunk_vel2.geom.shift(-1)
chunk_vel2 = chunk_vel2[:-1]
chunk_vel2.loc[:,'next_loc'] = chunk_vel2.next_loc.apply(lambda x : (x.x,x.y))

myradians = np.arctan2(chunk_vel2.current_loc.apply(lambda x: x[1]).values-chunk_vel2.next_loc.apply(lambda x: x[1]).values, chunk_vel2.current_loc.apply(lambda x: x[0]).values-chunk_vel2.next_loc.apply(lambda x: x[0]).values)
chunk_vel2['bearing'] = np.degrees(myradians)